In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# read source csv file, which is named as Public_toilet
df = pd.read_csv("Public_toilet.csv")

# select public toilets in victoria
df = df[df["State"] == "Victoria"]

# select public toilets located in melbourne CBD and some near monash caulfield
df_1 = df.loc[df['Town'].isin(['Melbourne','Carlton', 'Flemington', 'Carlton North','South Yarra', 
                               'Southbank','Docklands','Port Melbourne','West Melbourne', 
                              'North Melbourne', 'Kensington', 'Parkville','Glen Iris','Caulfield','Caulfield South',
                               'Caulfield East','Glen Huntly', 'Caulfield North', 'Malvern', 'Malvern East','Carnegie',
                              'Ormond','Murrumbeena','Chadstone'])]

# derive a primary key for the json file
df_1 = df_1.reset_index()
df_1['PrimaryKey'] = df_1.index + 1
df_1['PrimaryKey'] = "P"+df_1.PrimaryKey.map(str)

# Derive disabled flag for the toilets
df_1['DisableFlag'] = np.where(df_1['IconAltText']=='Male and Female, or Unisex (Accessible)', 'Yes', 'No')

# Derive details and do cleaning
df_1 = df_1.fillna("Blank_tobereplaced")
df_1["Details"] = df_1.IsOpen.str.cat(others=[df_1.OpeningHoursSchedule, df_1.OpeningHoursNote], sep=' ')
df_1["Details"] = df_1["Details"].str.replace('Blank_tobereplaced',' ')
df_1["Details"] = df_1["Details"].str.replace('Variable',' ')
df_1["Details"] = df_1["Details"].str.replace('DaylightHours','Open DaylightHours')
df_1["Details"] = df_1["Details"].str.replace('AllHours','Open All Hours')
df_1.Details = df_1.Details.str.strip()

for i in range(len(df_1)):  
    if df_1['Details'][i] == '':  
        df_1['Details'][i] = 'No Information Available'

# drop unwanted columns
df_2 = df_1.drop(['URL', 'Address1', 'IconURL', 'Status', 'State', 'ToiletID', 'Address1', 
                'Town', 'ToiletType', 'DumpPoint', 'Male', 'Female', 'Unisex', 
                'FacilityType', 'PaymentRequired','KeyRequired', 'Parking', 'ParkingNote', 'Notes',
               'AccessLimited', 'AccessibleMale', 'AccessibleFemale', 'AccessibleUnisex', 'ParkingAccessible',
               'AccessibleParkingNote', 'Ambulant', 'LHTransfer', 'MLAK', 'RHTransfer', 'AdultChange',
               'ChangingPlaces','Postcode', 'index', 'AddressNote', 'AccessNote', 'AccessibleNote', 
                  'OpeningHoursNote', 'OpeningHoursSchedule','IconAltText', 'SanitaryDisposal', 'SharpsDisposal',
                 'DrinkingWater', 'Showers','BabyChange','IsOpen'], axis=1)

# Rename columns and derive category
df_2 = df_2.rename(columns = {'Name':'Description'})
df_2['Category'] = 'Public Toilet'

# Arrange columns
df_2 = df_2[['PrimaryKey', 'Description', 'Category','Latitude', 'Longitude', 'DisableFlag', 'Details']]

C:\Users\RakeshOM\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\RakeshOM\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
j = df_2.set_index('PrimaryKey').to_json(orient='index')

In [4]:
# save the dataframe to JSON file
with open("public_toilet.json", "w") as text_file:
    text_file.write("{}".format(j))